<h1> ANN: Artificial Neural Networks </h1>
<br>
<h4> What are ANNs ? </h4>
<p>Artificial neural networks, usually simply called neural networks or neural nets, are computing systems inspired by the biological neural networks that constitute animal brains.</p>

<img align='left' width='500' src='https://www.tutorialspoint.com/artificial_intelligence/images/atypical_ann.jpg'>
<img align='right' width='500' src='https://miro.medium.com/max/1400/1*rptqFUQTMiruWZWKm-WMqQ.png'>


<p align ='right'> It contains input layer which is used to provide inputs to the neural networks.<br> This is then moved on to the hidden layer which computes various things.<br>And based on those computations we get the output layer<br>  Which neurons of the output layer is turned on determines where its classifed. In simple terms ANN is just a fancy classifier  </p>

<br>
<br>
<h4> Steps of ANN </h4>
<p> Here are simplified steps related to ANN which should be understood before coding </p>
<ol>
    <li> Initalize with random weights and bias </li>
    <li> Compute z for all input layers </li>
    <li> Feed z into activation function </li>
    <li> Procced to forward propagation on all layers </li>
    <li> Compute the cost function /loss function and Back propagate </li>
    <li> During Back Propagate adjusts weights and biases </li>
    <li> Repeat steps 2 to 6 untill n epochs </li>
</ol>
<h2> Project 1. XOR Gate Predictor (Perceptron) </h2>

<h3> 0. Installing Dependencies </h3>

In [3]:
!pip install pandas
!pip install numpy

<h3> 1. Importing Dependencies </h3>

In [5]:
import pandas as pd
import random
import numpy as np 

<h3> 2. Create Dataset for XOR Gate  </h3>
Let's create a dataset for any number of input XOR Gate 

In [17]:
def generate_dataset(dims):

    def or_output(x):
        return x%2


    df=pd.DataFrame()
    for i in range(0,dims):
        while True: 
            A=[random.randrange(0,2) for i in range(0,2**dims)]
            if A.count(1)==(2**dims)/2:
                break
        df[str(i)]=A

    op=0
    for i in df.columns:
        op+=df[i]

    df['Y']=op
    df['Y']=df['Y'].apply(or_output)
    df
    return df 

dims=int(input('Enter number of dims: '))
data=generate_dataset(dims)
data

0    2
1    1
2    0
3    1
Name: Y, dtype: int64


,0,1,Y
0,1,1,0
1,0,1,1
2,0,0,0
3,1,0,1


<h3> 3. Perceptron Creation </h3>
    <p> Initially we take input and then multiple with respective weights and biases </p>

In [465]:
def generate_weights(data):
    n=len(data.columns)-1
    return [random.uniform(0,1) for i in range(n)],random.uniform(0,1)

def sigmoid(x):    
    return 1/(1+np.exp(-x))

def cost(P,A):
    return (P-A)**2

def d_cost(P,A):
    h=0.001
    return (cost(P+h,A)-cost(P,A))/h

def d_sigmoid(x):
    h=0.0001
    return (sigmoid(x+h)-sigmoid(x))/h


def train_model(data,weights,bias):
    
    features=data.drop(columns=['Y'])
    labels=data['Y'].to_list()

    #converting to numpy arrays
    features=np.array(features)
        
    weights=np.array(weights)
    bias=np.array(bias)

    for i in range(0,len(features)):
        
        z=weights*features[i]+bias
        GD=(d_cost(sigmoid(z),labels[i])*d_sigmoid(z))/len(features)

        weights=weights-alpha*GD*features[i]
        bias=bias-alpha*GD


    print(weights)
    print(bias)
    print('--'*60)
    
    for epochs in range(0,35000):
        z=np.dot(features,weights)+bias
        #print([sigmoid(i) for i in z])
        #print(labels)
        for index in range(len(z)):
            gradient_decent_=2*(sigmoid(z[index])-labels[index])*d_sigmoid(z[index])*(1/len(z))
            weights=np.array(weights)-alpha*np.array(gradient_decent_)*np.array(features[index])
            bias=np.array(bias)-alpha*np.array(gradient_decent_)
    
    print(weights)
    print(bias)
    return weights,bias

alpha=10**-3
weights,bias=generate_weights(data)
weights,bias=train_model(data,weights,bias)


[0.8025791  0.44732293]
[0.83976263 0.83976042]


In [468]:
print(weights)
print(bias)
print(sigmoid(np.dot(weights,[0,0])+bias))

[0.8025791  0.44732293]
[0.83976263 0.83976042]
[0.69841522 0.69841475]


<h3>4. Tensorflow development </h3>

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models

model=models.Sequential()
model.add(layers.Dense(64,activation='relu',input_dim=(data.drop(columns=['Y'])).shape[1]))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(128*2,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='sgd',metrics='accuracy')

X=np.array(data.drop(columns=['Y']))
Y=np.array(data['Y']).reshape(-1,1)
model.fit(X,Y,epochs=300)




Epoch 1/300
1/1 [==============================] - 0s 280ms/step - loss: 0.6947 - accuracy: 0.5000
Epoch 2/300
1/1 [==============================] - 0s 3ms/step - loss: 0.6943 - accuracy: 0.2500
Epoch 3/300
1/1 [==============================] - 0s 4ms/step - loss: 0.6939 - accuracy: 0.2500
Epoch 4/300
1/1 [==============================] - 0s 6ms/step - loss: 0.6935 - accuracy: 0.2500
Epoch 5/300
1/1 [==============================] - 0s 5ms/step - loss: 0.6930 - accuracy: 0.5000
Epoch 6/300
1/1 [==============================] - 0s 7ms/step - loss: 0.6926 - accuracy: 0.5000
Epoch 7/300
1/1 [==============================] - 0s 7ms/step - loss: 0.6922 - accuracy: 0.5000
Epoch 8/300
1/1 [==============================] - 0s 6ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 9/300
1/1 [==============================] - 0s 6ms/step - loss: 0.6914 - accuracy: 0.5000
Epoch 10/300
1/1 [==============================] - 0s 6ms/step - loss: 0.6910 - accuracy: 0.5000
Epoch 11/300
1/1 [=========

In [30]:

print(model.predict([[1,1]]))

1/1 [==============================] - 0s 31ms/step
[[0.46086556]]
